In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "6"
from torch.utils.data import DataLoader, Dataset
import torch
import torch.nn as nn
import torch.optim as optim
from datasets import load_dataset, DatasetDict
import torch.utils.data as data
from transformers import AutoTokenizer
import math
import copy
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
import torch.nn.functional as F
import time
from tqdm import tqdm
qa_dataset = load_dataset("maywell/ko_wikidata_QA", cache_dir="../../../data/jeongseokoh/dataset/")
tokenizer_ko = AutoTokenizer.from_pretrained("klue/bert-base")
# Train 데이터셋을 랜덤하게 섞습니다.
shuffled_dataset = qa_dataset['train'].shuffle(seed=42)

# 30,000개를 train용, 10,000개를 validation용으로 분리
#train_dataset = shuffled_dataset.select(range(30000))
validation_dataset = shuffled_dataset.select(range(30000, 40000))


def ko_tokenize_function(examples):
    # 영어(en)와 독일어(de) 텍스트에 대한 토크나이징 수행
    tokenized_inputs = tokenizer_ko(examples["instruction"], padding="max_length", truncation=True, max_length=256)
    # 타겟 토크나이저로 영어 텍스트 토크나이징
    with tokenizer_ko.as_target_tokenizer():
        tokenized_targets = tokenizer_ko(examples["output"], padding="max_length", truncation=True, max_length=256)
    
    # 모델 입력을 위한 tokenized_inputs 반환 및 레이블(labels)로 tokenized_targets["input_ids"] 설정
    tokenized_inputs["labels"] = tokenized_targets["input_ids"]
    return tokenized_inputs
def collate_fn(batch):
    input_ids = torch.tensor([item['input_ids'] for item in batch])
    labels = torch.tensor([item['labels'] for item in batch])
    return {"input_ids": input_ids, "labels": labels}
# 데이터셋에 토큰화 함수 적용
tokenized_datasets = shuffled_dataset.map(ko_tokenize_function, batched=True)
val_tokenized_datasets = validation_dataset.map(ko_tokenize_function, batched=True)

# DataLoader 재정의
dataloader = DataLoader(tokenized_datasets, batch_size=32, shuffle=True, collate_fn=collate_fn)
val_dataloader = DataLoader(val_tokenized_datasets, batch_size=32, shuffle=True, collate_fn=collate_fn)
test_dataloader = DataLoader(val_tokenized_datasets, batch_size=1, shuffle=True, collate_fn=collate_fn)

In [2]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"
        #print(f"d_model : {d_model}")
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads
        
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)
        
    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)
        attn_probs = torch.softmax(attn_scores, dim=-1)
        output = torch.matmul(attn_probs, V)
        return output
        
    def split_heads(self, x):
        batch_size, seq_length, d_model = x.size()
        return x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)
        
    def combine_heads(self, x):
        batch_size, _, seq_length, d_k = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model)
        
    def forward(self, Q, K, V, mask=None):
        #print(f"Q: {Q.shape}")

        Q = self.split_heads(self.W_q(Q))
        K = self.split_heads(self.W_k(K))
        V = self.split_heads(self.W_v(V))
        #print(Q.shape)
        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)
        output = self.W_o(self.combine_heads(attn_output))
        #print(f"Output: {output.shape}")
        return output
class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super(PositionWiseFeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))
    
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_length):
        super(PositionalEncoding, self).__init__()
        
        pe = torch.zeros(max_seq_length, d_model)
        position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))
        
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        self.register_buffer('pe', pe.unsqueeze(0))
        
    def forward(self, x):
        #print((x + self.pe[:, :x.size(1)]).shape)
        return x + self.pe[:, :x.size(1)]
## 1. CustomWeightLayer를 expand와 shrink로 나눈다. 
## 2-1. 768차원으로 늘려서 Encoder의 Self-attention에 넣는다.
## 2-2. 768차원으로 늘려서 바로 decoder의 Cross-attention에 넣는다. 
class CustomWeightLayer(nn.Module):
    def __init__(self, d_model, num_heads, tokens):
        super(CustomWeightLayer, self).__init__()
        self.d_model = d_model
        self.tokens = tokens
        self.out_dim1 = 3 * self.tokens - 1
        self.out_dim2 = self.d_model
        self.relu = nn.ReLU()

        # 가중치 및 바이어스 초기화를 더 효율적으로 수행
        self.expand_w = nn.Parameter(torch.randn(self.out_dim1, self.tokens) * 0.01)
        self.expand_bias = nn.Parameter(torch.zeros(self.out_dim1))
        self.feature1_w = nn.Parameter(torch.randn(self.out_dim2, self.d_model) * 0.01)
        self.feature1_bias = nn.Parameter(torch.zeros(self.out_dim2))
        self.shrink_w = nn.Parameter(torch.randn(self.tokens, self.out_dim1) * 0.01)
        self.shrink_bias = nn.Parameter(torch.zeros(self.tokens))
        self.feature2_w = nn.Parameter(torch.randn(self.d_model, self.out_dim2) * 0.01)
        self.feature2_bias = nn.Parameter(torch.zeros(self.d_model))
        # 가중치에서 특정 패턴을 0으로 설정
        self._init_weights()
    
    def _init_weights(self):
        # expand_w와 shrink_w의 특정 패턴을 0으로 초기화
        with torch.no_grad():
            for i in range(self.tokens):
                for j in range(self.tokens - 1):
                    self.expand_w.data[i - self.tokens + 1 + j, i] = 0.0
                    self.shrink_w.data[i, i - self.tokens + 1 + j] = 0.0

    def forward(self, x, mask=None):
        if mask is not None:
            mask = mask.squeeze(1).squeeze(1)  # 두 번째와 세 번째 차원을 제거
            mask = mask.unsqueeze(-1)  # 마지막 차원에 차원 추가
            x = x * mask.float()
        # 텐서 조작을 최소화하고, 불필요한 전치를 피함
        x = F.linear(x, self.feature1_w, self.feature1_bias)
        x = F.linear(x.transpose(1, 2), self.expand_w, self.expand_bias)
        x = self.relu(x)
        
        x = F.linear(x, self.shrink_w, self.shrink_bias).transpose(1, 2)
        x = F.linear(x, self.feature2_w, self.feature2_bias)

        return x

class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout, tokens):
        super(EncoderLayer, self).__init__()
        self.tokens = tokens
        self.custom = CustomWeightLayer(d_model, num_heads, self.tokens)
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, mask, last_layer=False):
        our_output = self.self_attn(x, x, x, mask)
        x = self.norm1(x + self.dropout(our_output))
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))
        return x
class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout, tokens):
        super(DecoderLayer, self).__init__()
        self.tokens = tokens
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.cross_attn = MultiHeadAttention(d_model, num_heads)
        #self.custom = CustomWeightLayer(d_model, num_heads, self.tokens-1)
        #self.adaptive_pool = nn.AdaptiveAvgPool1d(127)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, enc_output, src_mask, tgt_mask):
        
        attn_output = self.self_attn(x, x, x, tgt_mask)
        x = self.norm1(x + self.dropout(attn_output))
        #print(f"x.size: {x.shape}")
        enc_output =  torch.cat((enc_output, x), dim=1)
        #print(f"enc_output.size: {enc_output.shape}")
        attn_output = self.cross_attn(x, enc_output, enc_output, src_mask)
        #print(f"attn_output.size: {attn_output.shape}")
        x = self.norm2(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm3(x + self.dropout(ff_output))
        return x
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout):
        super(Transformer, self).__init__()
        self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)
        self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, max_seq_length)
        self.encoder_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout, max_seq_length) for _ in range(num_layers)])
        self.decoder_layers = nn.ModuleList([DecoderLayer(d_model, num_heads, d_ff, dropout, max_seq_length) for _ in range(num_layers)])
        self.num_layers = num_layers
        self.fc = nn.Linear(d_model, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def generate_mask(self, src, tgt):
        src_mask = (src != 0).unsqueeze(1).unsqueeze(2).to(device)
        tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(3).to(device)
        seq_length = tgt.size(1)
        nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length), diagonal=1)).bool().to(device)
        tgt_mask = tgt_mask & nopeak_mask
        #print(tgt_mask.shape)
        #print(src_mask.shape)
        # src_mask를 tgt_mask 크기에 맞게 확장
        src_mask_expanded = src_mask.expand(-1, -1, seq_length, -1)  # 마지막 차원을 유지하면서 세 번째 차원을 seq_length로 확장

        # 두 텐서를 마지막 차원을 기준으로 연결
        combined_mask = torch.cat((src_mask_expanded, tgt_mask), dim=3)  # 마지막 차원(3)을 기준으로 연결

        return src_mask, tgt_mask, combined_mask

    def forward(self, src, tgt):
        seq_length = tgt.size(1)
        src_mask, tgt_mask, combined_mask = self.generate_mask(src, tgt)
        src_embedded = self.dropout(self.positional_encoding(self.encoder_embedding(src)))
        tgt_embedded = self.dropout(self.positional_encoding(self.decoder_embedding(tgt)))
        
        enc_output = src_embedded
        last_layer = False
        for i, enc_layer in enumerate(self.encoder_layers):
            if i+1 == self.num_layers:
                last_layer = True
            else:
                last_layer = False
            enc_output = enc_layer(enc_output, src_mask, last_layer)

        dec_output = tgt_embedded
        for dec_layer in self.decoder_layers:
            dec_output = dec_layer(dec_output, enc_output, combined_mask, tgt_mask)

        output = self.fc(dec_output)
        return output

In [3]:
import wandb
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
src_vocab_size = tokenizer_ko.vocab_size
tgt_vocab_size = tokenizer_ko.vocab_size
d_model = 768  # BERT의 경우
num_heads = 8
num_layers = 6
d_ff = 2048
max_seq_length = 256
dropout = 0.1
wandb.init(
        # set the wandb project where this run will be logged
        project="Sparse_plane",
        
        # track hyperparameters and run metadata
        config={
            "learning_rate": 0.01,
            "architecture": "Scratch",
            "dataset": "de,en",
            "epochs": 10,
            }
    )
# 모델 초기화
transformer = Transformer(
    src_vocab_size=src_vocab_size,
    tgt_vocab_size=tgt_vocab_size,
    d_model=d_model,
    num_heads=num_heads,
    num_layers=num_layers,
    d_ff=d_ff,
    max_seq_length=max_seq_length,
    dropout=dropout
).to(device)
#model_state_dict = torch.load('../../../data/jeongseokoh/model/base_noise_de_en.pth')
#transformer.load_state_dict(model_state_dict)
# 손실 함수 및 옵티마이저 정의
criterion = nn.CrossEntropyLoss(ignore_index=tokenizer_ko.pad_token_id)
optimizer = optim.AdamW(transformer.parameters(), lr=0.0001, weight_decay=0.01)

# 모델 학습
transformer.train()


start_time = time.time()

for epoch in range(1, 11):  # 에폭 수 조정 가능
    total_loss = 0
    total_var = 0
    for batch in tqdm(dataloader, desc="Train Iteration"):
        src = batch['input_ids'].to(device)  # 장치로 이동
        tgt = batch['labels'].to(device)     # 장치로 이동

        optimizer.zero_grad()
        
        output = transformer(src, tgt[:, :-1])  # [32, 128, vocab_size], [32]
        
        # 교차 엔트로피 손실 계산
        # 교차 엔트로피 손실 계산
        basic_loss = criterion(output.reshape(-1, tgt_vocab_size), tgt[:, 1:].reshape(-1))

        # 분산의 영향을 조절하기 위한 가중치 인자
        #variance_weight = 0.01  # 이 값은 실험을 통해 최적화해야 함

        # 손실에서 분산의 가중치를 곱한 값을 빼서 새로운 손실 값을 계산
        # 여기서 max 함수를 사용하여 손실 값이 음수가 되지 않도록 합니다.
        #adjusted_loss = basic_loss + torch.max(0.2 - variances.mean(), torch.tensor(0.0))

        basic_loss.backward()
        optimizer.step()
        wandb.log({"Loss": basic_loss.item()})
        total_loss += basic_loss.item()
        #total_var += variances.mean()
        
    avg_loss = total_loss / len(dataloader)
    #avg_var = total_var / len(dataloader)
    print(f"Epoch: {epoch}, Avg Train Loss: {avg_loss:.4f}")


    checkpoint = {
        'epoch': epoch,
        'model_state_dict': transformer.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': avg_loss,
        # You can add more components here as needed.
    }
    torch.save(checkpoint, f'../../parameters/scratch/expand_att_ko_wiki_checkpoint_epoch_{epoch}.pth')
    total_loss = 0
    for batch in tqdm(val_dataloader, desc="Val Iteration"):
        src = batch['input_ids'].to(device)  # 장치로 이동
        tgt = batch['labels'].to(device)     # 장치로 이동

        with torch.no_grad():
            output = transformer(src, tgt[:, :-1])
            loss = criterion(output.reshape(-1, tgt_vocab_size), tgt[:, 1:].reshape(-1))
         
        total_loss += loss.item()

    avg_loss = total_loss / len(val_dataloader)
    print(f"Epoch: {epoch}, Avg Validation Loss: {avg_loss:.4f}")
    wandb.log({"Val_Loss": avg_loss})   
end_time = time.time()
print(f"총 시간: {end_time - start_time} sec")
torch.save(transformer.state_dict(), '../../parameters/scratch/expand_att_ko_wiki.pth')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: luke0112. Use `wandb login --relogin` to force relogin


Train Iteration: 100%|██████████| 4298/4298 [35:18<00:00,  2.03it/s]


Epoch: 1, Avg Train Loss: 4.5080


In [23]:
# 텐서 정의
tensor1 = torch.randn(32, 128, 768)
tensor2 = torch.randn(32, 1, 768)

# 두 텐서를 두 번째 차원을 기준으로 연결
result_tensor = torch.cat((tensor1, tensor2), dim=1)

#print(result_tensor)
print(result_tensor.shape)

torch.Size([32, 129, 768])


In [11]:
def predict(model, input_sequence, max_length=128, SOS_token=101, EOS_token=102):
    model.eval()
    # Ensure input_sequence is a tensor and reshape if necessary
    if not isinstance(input_sequence, torch.Tensor):
        input_sequence = torch.tensor(input_sequence, dtype=torch.long)
    input_sequence = input_sequence.unsqueeze(0)  # Add batch dimension if missing
    input_sequence = input_sequence.to(device)  # Move to the same device as the model

    y_input = torch.tensor([[SOS_token]], dtype=torch.long, device=device)
    with torch.no_grad():
        for _ in range(max_length):
            pred = model(input_sequence, y_input)
            next_item = pred[:, -1, :].topk(1)[1].view(-1).item()  # Get the last word's top prediction
            next_item = torch.tensor([[next_item]], device=device)
            #print(next_item)
            y_input = torch.cat((y_input, next_item), dim=1)
            if next_item.view(-1).item() == EOS_token:
                break

    return y_input.view(-1).tolist()
 
val_dataloader.dataset['labels'][1]
# Here we test some examples to observe how the model predicts

examples2 = test_dataloader.dataset['input_ids'][:10]
 
for idx, example in enumerate(examples2):
    result = predict(transformer, example)
    print(f"Example {idx}")
    print(f"Input: {example[1:-1]}")
    print(f"GoldenAnswer: {test_dataloader.dataset['en'][idx]}")
    print(f"ExpandAnswer: {tokenizer_ko.decode(result[1:])}")
    print()


Example 0
Input: [917, 2509, 88, 9712, 26280, 3841, 20640, 26897, 115, 303, 19630, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
GoldenAnswer: A group of men are loading cotton onto a truck
ExpandAnswer: a group of men are unloading a truck. [SEP]

Example 1
Input: [198, 980, 10640, 136, 50, 297, 15544, 2566, 115, 297, 507, 8716, 26914, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
GoldenAnswer: A man sle

In [5]:
import torch.nn.functional as F
import Transformer_Base
import Transformer_Expand

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
src_vocab_size = tokenizer_src.vocab_size
tgt_vocab_size = tokenizer_tgt.vocab_size
d_model = 768  # BERT의 경우
num_heads = 8
num_layers = 6
d_ff = 2048
max_seq_length = 128
dropout = 0.1

# 모델 초기화
transformer = Transformer_Expand.Transformer(
    src_vocab_size=src_vocab_size,
    tgt_vocab_size=tgt_vocab_size,
    d_model=d_model,
    num_heads=num_heads,
    num_layers=num_layers,
    d_ff=d_ff,
    max_seq_length=max_seq_length,
    dropout=dropout
).to(device)
model_state_dict = torch.load('../../parameters/scratch/base_expand_de_en_checkpoint_epoch_5.pth')
transformer.load_state_dict(model_state_dict['model_state_dict'])
# 모델 초기화
transformer2 = Transformer_Base.Transformer(
    src_vocab_size=src_vocab_size,
    tgt_vocab_size=tgt_vocab_size,
    d_model=d_model,
    num_heads=num_heads,
    num_layers=num_layers,
    d_ff=d_ff,
    max_seq_length=max_seq_length,
    dropout=dropout
).to(device)
model_state_dict = torch.load('../../parameters/scratch/base_attention_de_en_checkpoint_epoch_5.pth')
transformer2.load_state_dict(model_state_dict['model_state_dict'])
#criterion = nn.CrossEntropyLoss(ignore_index=tokenizer_tgt.pad_token_id)
def predict(model, input_sequence, max_length=128, SOS_token=101, EOS_token=102):
    model.eval()
    # Ensure input_sequence is a tensor and reshape if necessary
    if not isinstance(input_sequence, torch.Tensor):
        input_sequence = torch.tensor(input_sequence, dtype=torch.long)
    input_sequence = input_sequence.unsqueeze(0)  # Add batch dimension if missing
    input_sequence = input_sequence.to(device)  # Move to the same device as the model

    y_input = torch.tensor([[SOS_token]], dtype=torch.long, device=device)
    with torch.no_grad():
        for _ in range(max_length):
            pred = model(input_sequence, y_input)
            next_item = pred[:, -1, :].topk(1)[1].view(-1).item()  # Get the last word's top prediction
            next_item = torch.tensor([[next_item]], device=device)
            #print(next_item)
            y_input = torch.cat((y_input, next_item), dim=1)
            if next_item.view(-1).item() == EOS_token:
                break

    return y_input.view(-1).tolist()
 
val_dataloader.dataset['labels'][1]
# Here we test some examples to observe how the model predicts

examples2 = test_dataloader.dataset['input_ids'][:10]
 
for idx, example in enumerate(examples2):
    result = predict(transformer, example)
    result2 = predict(transformer2, example)
    print(f"Example {idx}")
    print(f"Input: {example[1:-1]}")
    print(f"GoldenAnswer: {test_dataloader.dataset['en'][idx]}")
    print(f"ExpandAnswer: {tokenizer_tgt.decode(result[1:])}")
    print(f"Base  Answer: {tokenizer_tgt.decode(result2[1:])}")
    print()


Example 0
Input: [917, 2509, 88, 9712, 26280, 3841, 20640, 26897, 115, 303, 19630, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
GoldenAnswer: A group of men are loading cotton onto a truck
ExpandAnswer: a group of men are putting tree on a truck. [SEP]
Base  Answer: a group of men are loading tree stump onto a truck. [SEP]

Example 1
Input: [198, 980, 10640, 136, 50, 297, 15544, 2566, 115, 297, 507, 8716, 26914, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,